In [8]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from core.utils import dump_dict_to_yaml
from hummingbot.strategy_v2.executors.position_executor.data_types import TrailingStop
from controllers.generic.stat_arb import StatArbConfig, GridLimitsConfig
from hummingbot.core.data_type.common import TradeType
from core.services.backend_api_client import BackendAPIClient
from controllers.directional_trading.macd_bb_custom import MACDBBCustomControllerConfig

from decimal import Decimal

# GENERATE STAT ARB STRATEGY (BASE PART) (On Progress)

# GENERATE MACDBB STRATEGY (BEST PART)

In [ ]:
# Controller configuration
connector_name = "bybit_perpetual"
trading_pair = "WAL-USDT"
candles_connector="bybit_perpetual"
candles_trading_pair="WAL-USDT"
interval = "1m"
fast_ma = 21
slow_ma = 42
signal_ma = 9
bb_length = 200
natr_length = 200
bb_std = 2
total_amount_quote = 1000
max_executors_per_side = 5
take_profit = 0.03
stop_loss = 0.02
trailing_stop_activation_price = 0.007
trailing_stop_trailing_delta = 0.003
time_limit = 60 * 60 * 5
cooldown_time = 60 * 60
leverage = 10
step_percentage = "0.001"
position_sizing_method="volatility"
distribution_method="fibonacci"

# Creating the instance of the configuration and the controller
best_part_config = MACDBBCustomControllerConfig(
    connector_name=connector_name,
    trading_pair=trading_pair,
    interval=interval,
    macd_fast=fast_ma,
    position_sizing_method=position_sizing_method,
    distribution_method=distribution_method,
    macd_slow=slow_ma,
    macd_signal=signal_ma,
    bb_length=bb_length,
    candles_connector=candles_connector,
    candles_trading_pair=candles_trading_pair,
    bb_std=bb_std,
    natr_length=natr_length,
    leverage=leverage,
    step_percentage=step_percentage,
    total_amount_quote=Decimal(total_amount_quote),
    take_profit=Decimal(take_profit),
    stop_loss=Decimal(stop_loss),
    trailing_stop=TrailingStop(activation_price=Decimal(trailing_stop_activation_price), trailing_delta=Decimal(trailing_stop_trailing_delta)),
    time_limit=time_limit,
    max_executors_per_side=max_executors_per_side,
    cooldown_time=cooldown_time,
)

# EXPORT CONFIG TO YAML FILE

In [ ]:

# Path to save the configuration
save_to = root_path + '/config'

# Dump the configuration to a dictionary
best_config_json = best_part_config.model_dump(mode="json", exclude_none=True)
# base_config_json = base_part_config.model_dump(mode="json", exclude_none=True)
# Add the ID to the configuration that will used as file name
# base_config_json['id'] = f"base_part_stat_arb_{base_trading_pair}_{quote_trading_pair}"
best_config_json['id'] = f"best_part_macd_bb_{trading_pair}"
# Save the configuration to a YAML file
# dump_dict_to_yaml(config=base_config_json, folder=save_to)
dump_dict_to_yaml(config=best_config_json, folder=save_to)

# RUN STRATEGY

In [ ]:
# Use the backend API client to add the strategy
client = BackendAPIClient()
# The total balance that will used to run the asym_babel strategy
total_quote_balance = 100

base_part_balance_usage_percentage = 0.8
best_part_balance_usage_percentage = 0.2

configs = [best_config_json]

# Modified total amount quote balance for each strategy based on the asym_babel_each_strat_balance
for config in configs:
     quote_balance_multipliers = base_part_balance_usage_percentage if ("base_part" in config["id"]) else best_part_balance_usage_percentage
     config["total_amount_quote"] = total_quote_balance * quote_balance_multipliers
     await client.add_controller_config(config)

In [ ]:
# Deploy the strategy with multiple controllers
#Notes: Before running this script, make sure the image already has macd_bb_custom strategy

await client.deploy_script_with_controllers(
    bot_name="asym_babel",
    controller_configs=[config["id"] + ".yml" for config in configs],
    image_name="hummingbot/hummingbot:latest",
    max_global_drawdown=50,
    max_controller_drawdown=20,
)

{'success': True,
 'message': 'Instance hummingbot-asym_babel-2025.04.26_23.56 created successfully.'}